In [16]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
 

In [17]:
df = pd.read_csv('D:\\Personal projects\\House prices advanced Regression\\train.csv')

In [18]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split

# Target
y = df["SalePrice"]

# Features
X = df.drop(columns=["SalePrice", "Id"])
y_log = np.log1p(y)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y_log, 
                                                  test_size=0.2, 

                                                  random_state=42)

In [21]:
# performing a sanity check
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)


(1168, 79) (292, 79)
(1168,) (292,)


In [22]:
y_train.describe()

count    1168.000000
mean       12.030658
std         0.390606
min        10.460271
25%        11.775297
50%        12.013707
75%        12.278049
max        13.521141
Name: SalePrice, dtype: float64

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [24]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns
print("Numerical columns:", len(num_cols) , "Categorical:  " , len(cat_cols))

Numerical columns: 36 Categorical:   43


In [25]:
!pip install pytorch-tabnet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
numerric_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
pre_processor = ColumnTransformer(transformers=[
    ('num', numerric_pipe, num_cols),
    ('cat', categorical_pipe, cat_cols)

], remainder="drop")
# Fit on train only (prevents leakage)
X_train_pp = pre_processor.fit_transform(X_train)
X_val_pp   = pre_processor.transform(X_val)


# TabNet needs dense float32 arrays
X_train_pp = X_train_pp.toarray() if hasattr(X_train_pp, "toarray") else X_train_pp
X_val_pp   = X_val_pp.toarray() if hasattr(X_val_pp, "toarray") else X_val_pp

X_train_pp = X_train_pp.astype(np.float32)
X_val_pp   = X_val_pp.astype(np.float32)

y_train_np = y_train.values.reshape(-1, 1).astype(np.float32)  # log target
y_val_np   = y_val.values.reshape(-1, 1).astype(np.float32)

print(X_train_pp.shape, X_val_pp.shape, y_train_np.shape, y_val_np.shape)

(1168, 285) (292, 285) (1168, 1) (292, 1)


In [29]:
# --- TabNet: model definition + training + evaluation (fixed) ---

import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

tabnet_gpu = TabNetRegressor(
    n_d=32,
    n_a=32,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-2),
    mask_type="entmax",
    device_name=DEVICE
)

tabnet_gpu.fit(
    X_train_pp, y_train_np,
    eval_set=[(X_val_pp, y_val_np)],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=30,
    batch_size=2048,
    virtual_batch_size=256,
    num_workers=0,
    drop_last=False
)

# ---- Evaluation (use the trained model: tabnet_gpu) ----
val_pred_log = tabnet_gpu.predict(X_val_pp).reshape(-1)

y_val_usd = np.expm1(y_val)
val_pred_usd = np.expm1(val_pred_log)

rmse_usd = np.sqrt(mean_squared_error(y_val_usd, val_pred_usd))
mae_usd  = mean_absolute_error(y_val_usd, val_pred_usd)
mape     = np.mean(np.abs((y_val_usd - val_pred_usd) / y_val_usd)) * 100

print(f"RMSE (USD): ${rmse_usd:,.0f}")
print(f"MAE  (USD): ${mae_usd:,.0f}")
print(f"MAPE: {mape:.2f}%")


Using device: cuda


c:\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 147.7285| val_0_rmse: 11.6184 |  0:00:00s
epoch 1  | loss: 132.49654| val_0_rmse: 11.34497|  0:00:00s
epoch 2  | loss: 121.20547| val_0_rmse: 11.14136|  0:00:00s
epoch 3  | loss: 109.39629| val_0_rmse: 10.91993|  0:00:00s
epoch 4  | loss: 98.82285| val_0_rmse: 10.66445|  0:00:00s
epoch 5  | loss: 86.93841| val_0_rmse: 10.39225|  0:00:01s
epoch 6  | loss: 79.15044| val_0_rmse: 10.13826|  0:00:01s
epoch 7  | loss: 74.23303| val_0_rmse: 9.84275 |  0:00:01s
epoch 8  | loss: 65.72555| val_0_rmse: 9.51688 |  0:00:01s
epoch 9  | loss: 58.35172| val_0_rmse: 9.17883 |  0:00:01s
epoch 10 | loss: 53.68911| val_0_rmse: 8.79881 |  0:00:01s
epoch 11 | loss: 46.80884| val_0_rmse: 8.42198 |  0:00:02s
epoch 12 | loss: 44.1147 | val_0_rmse: 8.04785 |  0:00:02s
epoch 13 | loss: 38.03884| val_0_rmse: 7.68657 |  0:00:02s
epoch 14 | loss: 36.63087| val_0_rmse: 7.32319 |  0:00:02s
epoch 15 | loss: 30.59234| val_0_rmse: 6.93193 |  0:00:02s
epoch 16 | loss: 29.72782| val_0_rmse: 6.50919 |  0:0

c:\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


NameError: name 'tabnet' is not defined

In [30]:
import numpy as np
import optuna
import torch
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

def evaluate_metrics(model):
    pred_log = model.predict(X_val_pp).reshape(-1)

    rmse_log = np.sqrt(mean_squared_error(y_val, pred_log))

    y_val_usd = np.expm1(y_val)
    pred_usd  = np.expm1(pred_log)

    rmse_usd = np.sqrt(mean_squared_error(y_val_usd, pred_usd))
    mae_usd  = mean_absolute_error(y_val_usd, pred_usd)
    mape     = np.mean(np.abs((y_val_usd - pred_usd) / y_val_usd)) * 100

    return rmse_log, rmse_usd, mae_usd, mape


def objective(trial):
    params = {
        "n_d": trial.suggest_categorical("n_d", [16, 24, 32, 48, 64]),
        "n_a": trial.suggest_categorical("n_a", [16, 24, 32, 48, 64]),
        "n_steps": trial.suggest_int("n_steps", 3, 8),
        "gamma": trial.suggest_float("gamma", 1.2, 2.0),
        "n_independent": trial.suggest_int("n_independent", 1, 3),
        "n_shared": trial.suggest_int("n_shared", 1, 3),
        "lambda_sparse": trial.suggest_categorical("lambda_sparse", [0.0, 1e-6, 1e-5, 1e-4, 1e-3]),
        "mask_type": trial.suggest_categorical("mask_type", ["entmax", "sparsemax"]),
        "lr": trial.suggest_float("lr", 3e-3, 2e-2, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [512, 1024, 2048]),
        "virtual_batch_size": trial.suggest_categorical("virtual_batch_size", [64, 128, 256]),
    }

    model = TabNetRegressor(
        n_d=params["n_d"],
        n_a=params["n_a"],
        n_steps=params["n_steps"],
        gamma=params["gamma"],
        n_independent=params["n_independent"],
        n_shared=params["n_shared"],
        lambda_sparse=params["lambda_sparse"],
        mask_type=params["mask_type"],
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=params["lr"]),
        device_name=DEVICE,
    )

    # IMPORTANT: removed verbose=0 for compatibility with your pytorch-tabnet version
    model.fit(
        X_train_pp, y_train_np,
        eval_set=[(X_val_pp, y_val_np)],
        eval_metric=["rmse"],
        max_epochs=200,
        patience=25,
        batch_size=params["batch_size"],
        virtual_batch_size=params["virtual_batch_size"],
        num_workers=0,
        drop_last=False
    )

    rmse_log, rmse_usd, mae_usd, mape = evaluate_metrics(model)

    trial.set_user_attr("rmse_usd", float(rmse_usd))
    trial.set_user_attr("mae_usd", float(mae_usd))
    trial.set_user_attr("mape", float(mape))

    print(
        f"Trial {trial.number:03d} | rmse_log={rmse_log:.5f} | "
        f"RMSE$={rmse_usd:,.0f} | MAE$={mae_usd:,.0f} | MAPE={mape:.2f}% | "
        f"n_d/n_a={params['n_d']}/{params['n_a']} steps={params['n_steps']} "
        f"lr={params['lr']:.5f} batch={params['batch_size']} mask={params['mask_type']}"
    )

    del model
    if DEVICE == "cuda":
        torch.cuda.empty_cache()

    return rmse_log


pruner = optuna.pruners.MedianPruner(n_startup_trials=10)
study = optuna.create_study(direction="minimize", pruner=pruner)

N_TRIALS = 100  # bump to 200/500 for "crazy"
study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True, catch=(Exception,))

print("\n BEST RESULT")
print("Best rmse_log:", study.best_value)
print("Best params:", study.best_params)

df = study.trials_dataframe(attrs=("number", "value", "params", "state", "user_attrs"))
df = df.rename(columns={"value": "rmse_log"}).sort_values("rmse_log").reset_index(drop=True)
df.to_csv("optuna_tabnet_trials.csv", index=False)

print("\nTop 5 trials:")
print(df.head(5)[["number", "rmse_log", "user_attrs_rmse_usd", "user_attrs_mae_usd", "user_attrs_mape"]])

# Retrain best model (optional but useful)
bp = study.best_params
best_model = TabNetRegressor(
    n_d=bp["n_d"], n_a=bp["n_a"],
    n_steps=bp["n_steps"], gamma=bp["gamma"],
    n_independent=bp["n_independent"], n_shared=bp["n_shared"],
    lambda_sparse=bp["lambda_sparse"], mask_type=bp["mask_type"],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=bp["lr"]),
    device_name=DEVICE
)

best_model.fit(
    X_train_pp, y_train_np,
    eval_set=[(X_val_pp, y_val_np)],
    eval_metric=["rmse"],
    max_epochs=250,
    patience=30,
    batch_size=bp["batch_size"],
    virtual_batch_size=bp["virtual_batch_size"],
    num_workers=0,
    drop_last=False
)

rmse_log, rmse_usd, mae_usd, mape = evaluate_metrics(best_model)

print("\n FINAL BEST MODEL PERFORMANCE")
print(f"RMSE log : {rmse_log:.5f}")
print(f"RMSE USD : ${rmse_usd:,.0f}")
print(f"MAE  USD : ${mae_usd:,.0f}")
print(f"MAPE     : {mape:.2f}%")


[I 2025-12-28 13:03:09,985] A new study created in memory with name: no-name-aea485b3-defc-48aa-b5cb-6e82bd95346e


Using device: cuda


c:\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 236.94077| val_0_rmse: 12.36984|  0:00:00s
epoch 1  | loss: 215.0728| val_0_rmse: 12.07688|  0:00:00s
epoch 2  | loss: 201.21378| val_0_rmse: 11.87047|  0:00:00s
epoch 3  | loss: 182.64177| val_0_rmse: 11.70593|  0:00:01s
epoch 4  | loss: 168.3806| val_0_rmse: 11.56545|  0:00:01s
epoch 5  | loss: 155.49223| val_0_rmse: 11.40629|  0:00:01s
epoch 6  | loss: 140.72847| val_0_rmse: 11.24254|  0:00:01s
epoch 7  | loss: 130.78879| val_0_rmse: 11.08058|  0:00:02s
epoch 8  | loss: 118.72376| val_0_rmse: 10.88294|  0:00:02s
epoch 9  | loss: 108.43079| val_0_rmse: 10.67912|  0:00:02s
epoch 10 | loss: 94.71946| val_0_rmse: 10.42032|  0:00:03s
epoch 11 | loss: 86.02881| val_0_rmse: 10.10105|  0:00:03s
epoch 12 | loss: 78.40481| val_0_rmse: 9.80875 |  0:00:03s
epoch 13 | loss: 68.4954 | val_0_rmse: 9.46372 |  0:00:03s
epoch 14 | loss: 63.39425| val_0_rmse: 9.2287  |  0:00:04s
epoch 15 | loss: 55.98323| val_0_rmse: 8.94937 |  0:00:04s
epoch 16 | loss: 49.75786| val_0_rmse: 8.63064 |

c:\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-28 13:04:00,507] Trial 0 finished with value: 0.3612857290465453 and parameters: {'n_d': 64, 'n_a': 48, 'n_steps': 5, 'gamma': 1.997060055576506, 'n_independent': 3, 'n_shared': 2, 'lambda_sparse': 1e-06, 'mask_type': 'sparsemax', 'lr': 0.005993026166684755, 'batch_size': 2048, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.3612857290465453.


...

c:\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-12-28 13:52:00,306] Trial 99 finished with value: 0.20959556667921062 and parameters: {'n_d': 48, 'n_a': 24, 'n_steps': 3, 'gamma': 1.2482312175087258, 'n_independent': 2, 'n_shared': 3, 'lambda_sparse': 0.001, 'mask_type': 'entmax', 'lr': 0.010685443110938659, 'batch_size': 2048, 'virtual_batch_size': 128}. Best is trial 75 with value: 0.1600434375972372.


Trial 099 | rmse_log=0.20960 | RMSE$=38,876 | MAE$=26,268 | MAPE=16.33% | n_d/n_a=48/24 steps=3 lr=0.01069 batch=2048 mask=entmax

 BEST RESULT
Best rmse_log: 0.1600434375972372
Best params: {'n_d': 24, 'n_a': 16, 'n_steps': 3, 'gamma': 1.867196997241307, 'n_independent': 2, 'n_shared': 3, 'lambda_sparse': 0.0001, 'mask_type': 'entmax', 'lr': 0.01777098127657113, 'batch_size': 512, 'virtual_batch_size': 64}

Top 5 trials:
   number  rmse_log  user_attrs_rmse_usd  user_attrs_mae_usd  user_attrs_mape
0      75  0.160043         32632.965394        20026.878585        11.768393
1      93  0.169799         34613.043894        23053.024173        13.006341
2      96  0.174254         38834.707081        22019.774026        12.356991
3      13  0.175026         35752.897625        21563.782588        12.861022
4      57  0.175271         32794.449779        22151.293504        13.154037


c:\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 124.04973| val_0_rmse: 10.99165|  0:00:00s
epoch 1  | loss: 89.06008| val_0_rmse: 9.9431  |  0:00:00s
epoch 2  | loss: 59.40515| val_0_rmse: 8.74337 |  0:00:00s
epoch 3  | loss: 38.15757| val_0_rmse: 7.40508 |  0:00:00s
epoch 4  | loss: 23.74859| val_0_rmse: 5.91822 |  0:00:01s
epoch 5  | loss: 14.49446| val_0_rmse: 4.49727 |  0:00:01s
epoch 6  | loss: 11.89999| val_0_rmse: 3.49597 |  0:00:01s
epoch 7  | loss: 9.40919 | val_0_rmse: 3.2735  |  0:00:01s
epoch 8  | loss: 5.12708 | val_0_rmse: 3.25676 |  0:00:02s
epoch 9  | loss: 3.27222 | val_0_rmse: 2.73516 |  0:00:02s
epoch 10 | loss: 2.31068 | val_0_rmse: 1.89623 |  0:00:02s
epoch 11 | loss: 1.48752 | val_0_rmse: 1.91827 |  0:00:02s
epoch 12 | loss: 0.98775 | val_0_rmse: 1.75224 |  0:00:03s
epoch 13 | loss: 0.87301 | val_0_rmse: 1.15747 |  0:00:03s
epoch 14 | loss: 0.99645 | val_0_rmse: 1.19672 |  0:00:03s
epoch 15 | loss: 0.72089 | val_0_rmse: 1.09848 |  0:00:03s
epoch 16 | loss: 0.60768 | val_0_rmse: 0.87898 |  0:00:

c:\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
